In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ORRFA import *
import pandas as pd
import interpretableai
from interpretableai import iai
import pandas as pd
import json
from HR import *
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

  HTTP.Exceptions.RequestError(HTTP.Messages.Request:
"""
GET /versions.json HTTP/1.1
Host: docs.interpretable.ai
Accept: */*
User-Agent: HTTP.jl/1.8.1
Content-Length: 0
Accept-Encoding: gzip

""", EOFError())


In [2]:
dataset = pd.read_csv("/Users/ryanlucas/Desktop/kc_house_data.csv")

In [3]:
features, diagnosis, features_test, diagnosis_test = data_processing(dataset)

In [4]:
print(len(dataset))
print(len(features))
print(len(features_test))

21613
15129
6484


In [5]:
# Fit the regressor, set max_depth = 3

regr = DecisionTreeRegressor(max_depth=4, random_state=1234)
model = regr.fit(features, diagnosis)
# Print rules
dt_rules = get_rules(model,features.columns,"class_names", features)

In [6]:
dt_rules

["(features['grade'] < 0.708) & (features['lat'] >= -0.192) & (features['sqft_living'] < -0.059) & (features['sqft_living'] >= -0.689)",
 "(features['grade'] < 0.708) & (features['lat'] >= -0.192) & (features['sqft_living'] >= -0.059) & (features['waterfront'] < 5.524)",
 "(features['grade'] < 0.708) & (features['lat'] >= -0.192) & (features['sqft_living'] < -0.059) & (features['sqft_living'] < -0.689)",
 "(features['grade'] >= 0.708) & (features['sqft_living'] < 2.272) & (features['lat'] >= -0.315) & (features['yr_built'] >= 0.076)",
 "(features['grade'] < 0.708) & (features['lat'] < -0.192) & (features['sqft_living'] < -0.164) & (features['sqft_living'] < -0.702)",
 "(features['grade'] < 0.708) & (features['lat'] < -0.192) & (features['sqft_living'] < -0.164) & (features['sqft_living'] >= -0.702)",
 "(features['grade'] < 0.708) & (features['lat'] < -0.192) & (features['sqft_living'] >= -0.164) & (features['lat'] >= -1.056)",
 "(features['grade'] < 0.708) & (features['lat'] < -0.192) 

In [7]:
dt_df = pd.DataFrame()
for rule in dt_rules:
    index_feature = features.loc[eval(rule)].index
    new_feature = [1 if i in index_feature else 0 for i in range(len(features))]
    dt_df[rule] = new_feature

In [8]:
dt_df

,(features['grade'] < 0.708) & (features['lat'] >= -0.192) & (features['sqft_living'] < -0.059) & (features['sqft_living'] >= -0.689),(features['grade'] < 0.708) & (features['lat'] >= -0.192) & (features['sqft_living'] >= -0.059) & (features['waterfront'] < 5.524),(features['grade'] < 0.708) & (features['lat'] >= -0.192) & (features['sqft_living'] < -0.059) & (features['sqft_living'] < -0.689),(features['grade'] >= 0.708) & (features['sqft_living'] < 2.272) & (features['lat'] >= -0.315) & (features['yr_built'] >= 0.076),(features['grade'] < 0.708) & (features['lat'] < -0.192) & (features['sqft_living'] < -0.164) & (features['sqft_living'] < -0.702),(features['grade'] < 0.708) & (features['lat'] < -0.192) & (features['sqft_living'] < -0.164) & (features['sqft_living'] >= -0.702),(features['grade'] < 0.708) & (features['lat'] < -0.192) & (features['sqft_living'] >= -0.164) & (features['lat'] >= -1.056),(features['grade'] < 0.708) & (features['lat'] < -0.192) & (features['sqft_living'] >= -0.164) & (features['lat'] < -1.056),(features['grade'] >= 0.708) & (features['sqft_living'] < 2.272) & (features['lat'] < -0.315) & (features['waterfront'] < 5.524),(features['grade'] >= 0.708) & (features['sqft_living'] < 2.272) & (features['lat'] >= -0.315) & (features['yr_built'] < 0.076),(features['grade'] >= 0.708) & (features['sqft_living'] >= 2.272) & (features['sqft_living'] < 6.326) & (features['long'] >= 0.089),(features['grade'] >= 0.708) & (features['sqft_living'] >= 2.272) & (features['sqft_living'] < 6.326) & (features['long'] < 0.089),(features['grade'] >= 0.708) & (features['sqft_living'] < 2.272) & (features['lat'] < -0.315) & (features['waterfront'] >= 5.524),(features['grade'] < 0.708) & (features['lat'] >= -0.192) & (features['sqft_living'] >= -0.059) & (features['waterfront'] >= 5.524),(features['grade'] >= 0.708) & (features['sqft_living'] >= 2.272) & (features['sqft_living'] >= 6.326) & (features['long'] < 0.096),(features['grade'] >= 0.708) & (features['sqft_living'] >= 2.272) & (features['sqft_living'] >= 6.326) & (features['long'] >= 0.096)
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
15125,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
15126,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
15127,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
dt_df = dt_df.loc[:,~dt_df.columns.duplicated()].copy()

np.random.seed(1)
rules = {}

# for j in range(2, 7):

for j in [5]:

    for i in range(1,51):

        n = max(5, j+1)

        #################
        sampled_features = features.copy().sample(n=int(n),axis='columns')

        OCT_H = iai.GridSearch(
                  iai.OptimalTreeRegressor(
                      max_depth = j, cp = 0.00000000001))

        OCT_H.fit(sampled_features, diagnosis)
        OCT_H.write_json(f"tre.json")

        ######################
        f = open(f"tre.json")
        data = json.load(f)

        paths = gen_paths(data)

        sub_paths = gen_subpaths(paths)

        names = sampled_features.columns

        rule_features_new = gen_features(
                    sub_paths, data, sampled_features, names)

        rules[f"tre_{i}_{j}"] = rule_features_new

        print(f"Tree: {i}")

big_df = pd.DataFrame()
for tree in rules.keys():
    big_df = pd.concat([big_df, rules[tree]],axis =1)


In [10]:
generated_features = dt_df.loc[:,~dt_df.columns.duplicated()].copy()

for column in generated_features.columns:
    if generated_features[column].sum() == len(generated_features):
        del generated_features[column]

In [11]:
def gen_test_features(features_test, generated_features):
    features = features_test
    for rule in generated_features.columns:

        print(rule)

        index_feature = list(features.loc[eval(rule)].index)

        new_feature = [1 if i in index_feature else 0 for i in range(features.index[0], features.index[-1]+1)]

        features[rule] = new_feature

    return features_test

In [12]:
def reg_results(generated_features, features, diagnosis, features_test, diagnosis_test, gen = True):
    if gen:
        features = pd.concat([features, generated_features], axis = 1)

    if "Intercept" not in features.columns:
        features.insert(loc = 0, column = 'Intercept', value = [1 for i in range(len(features))])

    reg = linear_model.Ridge(alpha = 1).fit(features.copy()
, diagnosis)
    predictions_baseline = reg.predict(features.copy())

    SSE_model = sum((diagnosis - predictions_baseline)**2)
    SSE_mean = sum((diagnosis - diagnosis.mean())**2)

    IS_R2 = 1 - SSE_model/SSE_mean

    if "Intercept" not in features_test.columns:
        features_test.insert(loc = 0, column = 'Intercept', value = [1 for i in range(len(features_test))])

    if gen:
        features_test = gen_test_features(features_test, generated_features)
    predictions_test = reg.predict(features_test.copy())

    SSE_model_test = sum((diagnosis_test - predictions_test)**2)
    SSE_mean_test = sum((diagnosis_test - diagnosis.mean())**2)

    OOS_R2 = 1 - SSE_model_test/SSE_mean_test

    return IS_R2, OOS_R2

In [13]:
reg_results(dt_df, features, diagnosis, features_test, diagnosis_test, gen= True)

(features['grade'] < 0.708) & (features['lat'] >= -0.192) & (features['sqft_living'] < -0.059) & (features['sqft_living'] >= -0.689)
(features['grade'] < 0.708) & (features['lat'] >= -0.192) & (features['sqft_living'] >= -0.059) & (features['waterfront'] < 5.524)
(features['grade'] < 0.708) & (features['lat'] >= -0.192) & (features['sqft_living'] < -0.059) & (features['sqft_living'] < -0.689)
(features['grade'] >= 0.708) & (features['sqft_living'] < 2.272) & (features['lat'] >= -0.315) & (features['yr_built'] >= 0.076)
(features['grade'] < 0.708) & (features['lat'] < -0.192) & (features['sqft_living'] < -0.164) & (features['sqft_living'] < -0.702)
(features['grade'] < 0.708) & (features['lat'] < -0.192) & (features['sqft_living'] < -0.164) & (features['sqft_living'] >= -0.702)
(features['grade'] < 0.708) & (features['lat'] < -0.192) & (features['sqft_living'] >= -0.164) & (features['lat'] >= -1.056)
(features['grade'] < 0.708) & (features['lat'] < -0.192) & (features['sqft_living'] >= 

(0.7873962756735616, 0.7625679273365368)